In [5]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


----
---
---
---
---
---


# 92.92 % ACCURACY CODE.
## Total params: 995,230
---
---
---


# Oct 22 
---

# 1 - 55 Epochs

## Validation Accuracy - 87%

In [0]:

#keras library

import keras
import keras.backend as K
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam 
from keras.layers import Input
from keras.callbacks import LearningRateScheduler


# densenet 

def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp
  
  
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg
 




def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output
  
#############################################################################
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 55
l = 12 #L
num_filter = 36 #k
compression = 0.5
dropout_rate = 0.2
#############################################################################
  
  
  
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]




# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



#normalizing data

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255





input = Input(shape=(img_height, img_width, channel))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

# Back_Prop_First_Conv2D

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)


Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)



model = Model(inputs=[input], outputs=[output])
# model.summary()



#optimizer
sgd = keras.optimizers.SGD(lr=0.1 ,  momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

#callbacks


def lr_schedule(epoch):
  if epoch <= 150:
    K.set_value(model.optimizer.lr, 0.1)
    print("learning rate --->" , 0.1 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  elif epoch > 150 and epoch <=225:
    K.set_value(model.optimizer.lr, 0.01)
    print("learning rate --->" , 0.01 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  else:
    K.set_value(model.optimizer.lr, 0.001)
    print("learning rate --->" , 0.001 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
    
      
  
lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=5)


csv_logger = keras.callbacks.CSVLogger(filename =  "fifth__.csv", separator=',', append=False)


datagen = keras.preprocessing.image.ImageDataGenerator(
#     featurewise_std_normalization=True,
#     rotation_range=9 , 
    width_shift_range=0.125,
    height_shift_range=0.125,
    horizontal_flip=True)



datagen.fit(x_train)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    callbacks=[lr_reducer , csv_logger , model_checkpoint] ,
#                     steps_per_epoch= len(x_train)/batch_size,
                    epochs=epochs , 
                   verbose = 1 , 
                   workers = 4 , 
#                    use_multiprocessing = True , 
                   shuffle = True , 
                   validation_data=(x_test, y_test))



# 56 - 100 Epoch

## Validation Accuracy - 90%

In [0]:
model.load_weights("/gdrive/My Drive/CIFAR_10/weights.55-0.87.hdf5")

sgd = keras.optimizers.SGD(lr=0.1 ,  momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

#callbacks


def lr_schedule(epoch):
  
  epoch += 55 
  if epoch <= 150:
    K.set_value(model.optimizer.lr, 0.1)
    print("learning rate --->" , 0.1 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  elif epoch > 150 and epoch <=225:
    K.set_value(model.optimizer.lr, 0.01)
    print("learning rate --->" , 0.01 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  else:
    K.set_value(model.optimizer.lr, 0.001)
    print("learning rate --->" , 0.001 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
    
      
  
lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=5)


csv_logger = keras.callbacks.CSVLogger(filename =  "fifth__.csv", separator=',', append=False)


datagen = keras.preprocessing.image.ImageDataGenerator(
#     featurewise_std_normalization=True,
#     rotation_range=9 , 
    width_shift_range=0.125,
    height_shift_range=0.125,
    horizontal_flip=True)



datagen.fit(x_train)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    callbacks=[lr_reducer , csv_logger , model_checkpoint] ,
#                     steps_per_epoch= len(x_train)/batch_size,
                    epochs=45 , 
                   verbose = 1 , 
                   workers = 4 , 
#                    use_multiprocessing = True , 
                   shuffle = True , 
                   validation_data=(x_test, y_test))

# 100 - 150 Epoch

## Validation Accuracy - 91.99%

In [8]:
model.load_weights("/gdrive/My Drive/CIFAR_10/weights.45-0.90.hdf5")

sgd = keras.optimizers.SGD(lr=0.1 ,  momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

#callbacks


def lr_schedule(epoch):
  
  epoch += 100 
  if epoch <= 120:
    K.set_value(model.optimizer.lr, 0.1)
    print("learning rate --->" , 0.1 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  elif epoch > 120 and epoch <=210:
    K.set_value(model.optimizer.lr, 0.01)
    print("learning rate --->" , 0.01 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  else:
    K.set_value(model.optimizer.lr, 0.001)
    print("learning rate --->" , 0.001 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
    
      
  
lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=1)


csv_logger = keras.callbacks.CSVLogger(filename =  "fifth__.csv", separator=',', append=False)


datagen = keras.preprocessing.image.ImageDataGenerator(
#     featurewise_std_normalization=True,
#     rotation_range=9 , 
    width_shift_range=0.125,
    height_shift_range=0.125,
    horizontal_flip=True)



datagen.fit(x_train)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    callbacks=[lr_reducer , csv_logger , model_checkpoint] ,
#                     steps_per_epoch= len(x_train)/batch_size,
                    epochs=45 , 
                   verbose = 1 , 
                   workers = 4 , 
#                    use_multiprocessing = True , 
                   shuffle = True , 
                   validation_data=(x_test, y_test))

Epoch 1/45
learning rate ---> 0.1 Epoch ---> 100
782/782 [==============================] - 311s 398ms/step - loss: 0.2192 - acc: 0.9221 - val_loss: 0.3970 - val_acc: 0.8907

Epoch 00001: val_acc improved from -inf to 0.89070, saving model to /gdrive/My Drive/CIFAR_10/weights.01-0.89.hdf5
Epoch 2/45
learning rate ---> 0.1 Epoch ---> 101
782/782 [==============================] - 297s 380ms/step - loss: 0.2178 - acc: 0.9235 - val_loss: 0.4981 - val_acc: 0.8796

Epoch 00002: val_acc did not improve from 0.89070
Epoch 3/45
learning rate ---> 0.1 Epoch ---> 102
782/782 [==============================] - 297s 380ms/step - loss: 0.2166 - acc: 0.9241 - val_loss: 0.3381 - val_acc: 0.8995

Epoch 00003: val_acc improved from 0.89070 to 0.89950, saving model to /gdrive/My Drive/CIFAR_10/weights.03-0.90.hdf5
Epoch 4/45
learning rate ---> 0.1 Epoch ---> 103
782/782 [==============================] - 298s 381ms/step - loss: 0.2136 - acc: 0.9255 - val_loss: 0.3725 - val_acc: 0.9003

Epoch 00004: val_

# 150 - 180 Epochs

## Validation Accuracy - 92.19%

In [0]:
model.load_weights("/gdrive/My Drive/CIFAR_10/weights.45-0.92.hdf5")

sgd = keras.optimizers.SGD(lr=0.1 ,  momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

#callbacks


def lr_schedule(epoch):
  
  epoch += 150 
  if epoch <= 120:
    K.set_value(model.optimizer.lr, 0.1)
    print("learning rate --->" , 0.1 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  elif epoch > 120 and epoch <=210:
    K.set_value(model.optimizer.lr, 0.01)
    print("learning rate --->" , 0.01 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  else:
    K.set_value(model.optimizer.lr, 0.001)
    print("learning rate --->" , 0.001 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
    
      
  
lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=1)


csv_logger = keras.callbacks.CSVLogger(filename =  "fifth__.csv", separator=',', append=False)


datagen = keras.preprocessing.image.ImageDataGenerator(
#     featurewise_std_normalization=True,
#     rotation_range=9 , 
    width_shift_range=0.125,
    height_shift_range=0.125,
    horizontal_flip=True)



datagen.fit(x_train)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    callbacks=[lr_reducer , csv_logger , model_checkpoint] ,
#                     steps_per_epoch= len(x_train)/batch_size,
                    epochs=30 , 
                   verbose = 1 , 
                   workers = 4 , 
#                    use_multiprocessing = True , 
                   shuffle = True , 
                   validation_data=(x_test, y_test))

Epoch 1/30
learning rate ---> 0.01 Epoch ---> 150
782/782 [==============================] - 315s 402ms/step - loss: 0.1195 - acc: 0.9584 - val_loss: 0.3483 - val_acc: 0.9168

Epoch 00001: val_acc improved from -inf to 0.91680, saving model to /gdrive/My Drive/CIFAR_10/weights.01-0.92.hdf5
Epoch 2/30
learning rate ---> 0.01 Epoch ---> 151
782/782 [==============================] - 298s 381ms/step - loss: 0.1170 - acc: 0.9584 - val_loss: 0.3473 - val_acc: 0.9173

Epoch 00002: val_acc improved from 0.91680 to 0.91730, saving model to /gdrive/My Drive/CIFAR_10/weights.02-0.92.hdf5
Epoch 3/30
learning rate ---> 0.01 Epoch ---> 152
782/782 [==============================] - 298s 381ms/step - loss: 0.1156 - acc: 0.9589 - val_loss: 0.3588 - val_acc: 0.9165

Epoch 00003: val_acc did not improve from 0.91730
Epoch 4/30
learning rate ---> 0.01 Epoch ---> 153
782/782 [==============================] - 298s 381ms/step - loss: 0.1175 - acc: 0.9584 - val_loss: 0.3484 - val_acc: 0.9159

Epoch 00004: 

# Refresh

In [2]:
#keras library

import keras
import keras.backend as K
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam 
from keras.layers import Input
from keras.callbacks import LearningRateScheduler


# densenet 

def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp
  
  
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg
 




def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output
  
#############################################################################
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 200
l = 12 #L
num_filter = 36 #k
compression = 0.5
dropout_rate = 0.2
#############################################################################
  
  
  
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]




# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



#normalizing data

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255





input = Input(shape=(img_height, img_width, channel))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

# Back_Prop_First_Conv2D

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)


Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)



model = Model(inputs=[input], outputs=[output])
model.summary()


Using TensorFlow backend.


170500096/170498071 [==============================] - 44s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 36)   972         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 36)   144         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 36)   0           batch_normalization_1[0][0]      
_________________________________________

# Model Evaluate

In [5]:
sgd = keras.optimizers.SGD(lr=0.1 ,  momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.load_weights("/gdrive/My Drive/CIFAR_10/weights.17-0.92.hdf5")
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 24s 2ms/step
Test loss: 0.34242540892288087
Test accuracy: 0.9219


# Experimenting to increase accuracy

# Multiple version of models between 180-220. We will select the best.


## /gdrive/My Drive/CIFAR_10/weights.01-0.92.hdf5-92.25%
## /gdrive/My Drive/CIFAR_10/weights.04-0.92.hdf5-92.24%
##/gdrive/My Drive/CIFAR_10/__24__weights.01-0.92.hdf5-92.27%

In [3]:
import keras
import keras.backend as K
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam 
from keras.layers import Input
from keras.callbacks import LearningRateScheduler


# densenet 

def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp
  
  
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg
 




def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output
  
#############################################################################
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 200
l = 12 #L
num_filter = 36 #k
compression = 0.5
dropout_rate = 0.2
#############################################################################
  
  
  
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]




# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



#normalizing data

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255





input = Input(shape=(img_height, img_width, channel))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

# Back_Prop_First_Conv2D

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)


Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)



model = Model(inputs=[input], outputs=[output])
model.summary()


170500096/170498071 [==============================] - 37s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 36)   972         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 36)   144         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 36)   0           batch_normalization_1[0][0]      
_________________________________________

# 180 - 220 Epochs --------> Accuracy - 92.92%

In [7]:

model.load_weights("/gdrive/My Drive/CIFAR_10/weights.17-0.92.hdf5")
sgd = keras.optimizers.SGD(lr=0.01 ,decay = 1e-6 ,   momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])



reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                              factor=0.1, 
                                              patience=1, 
                                              verbose=1, 
                                              mode='auto', 
#                                               min_delta=0.0001, 
#                                               cooldown=0, 
                                              min_lr=0.001)

      
  
# lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/__24__weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=1)


csv_logger = keras.callbacks.CSVLogger(filename =  "180-250-epoch__.csv", separator=',', append=False)


datagen = keras.preprocessing.image.ImageDataGenerator(
#     featurewise_std_normalization=True,
#     rotation_range=9 , 
#     width_shift_range=0.125,
#     height_shift_range=0.125,
    horizontal_flip=True)



datagen.fit(x_train)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=128),
                    callbacks=[ csv_logger , model_checkpoint] ,
#                     steps_per_epoch= len(x_train)/batch_size,
                    epochs= 40 , 
                   verbose = 1 , 
                   workers = 4 , 
#                    use_multiprocessing = True , 
                   shuffle = True , 
                   validation_data=(x_test, y_test))




Epoch 1/40
391/391 [==============================] - 276s 705ms/step - loss: 0.0772 - acc: 0.9726 - val_loss: 0.3066 - val_acc: 0.9263

Epoch 00001: val_acc improved from -inf to 0.92630, saving model to /gdrive/My Drive/CIFAR_10/__24__weights.01-0.93.hdf5
Epoch 2/40
391/391 [==============================] - 262s 670ms/step - loss: 0.0745 - acc: 0.9739 - val_loss: 0.3076 - val_acc: 0.9272

Epoch 00002: val_acc improved from 0.92630 to 0.92720, saving model to /gdrive/My Drive/CIFAR_10/__24__weights.02-0.93.hdf5
Epoch 3/40
391/391 [==============================] - 262s 671ms/step - loss: 0.0708 - acc: 0.9752 - val_loss: 0.3173 - val_acc: 0.9258

Epoch 00003: val_acc did not improve from 0.92720
Epoch 4/40
391/391 [==============================] - 263s 673ms/step - loss: 0.0687 - acc: 0.9763 - val_loss: 0.3191 - val_acc: 0.9255

Epoch 00004: val_acc did not improve from 0.92720
Epoch 5/40
391/391 [==============================] - 263s 672ms/step - loss: 0.0702 - acc: 0.9754 - val_lo

# Model Evaluate

In [6]:
import keras

sgd = keras.optimizers.SGD(lr=0.01 ,  momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.load_weights("/gdrive/My Drive/CIFAR_10/__24__weights.17-0.93.hdf5")
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 25s 2ms/step
Test loss: 0.32899246576428415
Test accuracy: 0.9292


# 10 more Epochs remaining to get 94%

# Remove Augmentation for 1 Epoch.

# 9266

In [8]:

model.load_weights("/gdrive/My Drive/CIFAR_10/__24__weights.17-0.93.hdf5")
sgd = keras.optimizers.SGD(lr=0.01 ,decay = 1e-6 ,   momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])



reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                              factor=0.1, 
                                              patience=1, 
                                              verbose=1, 
                                              mode='auto', 
#                                               min_delta=0.0001, 
#                                               cooldown=0, 
                                              min_lr=0.001)

      
  
# lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/__24__weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=1)


csv_logger = keras.callbacks.CSVLogger(filename =  "180-250-epoch__.csv", separator=',', append=False)


model.fit(x=x_train, 
          y= y_train, 
          callbacks=[ csv_logger , model_checkpoint],
          batch_size=64, 
          epochs=1, 
          verbose=1, 
          validation_split=0.0, 
          validation_data=(x_test, y_test), 
          shuffle=True)
         



Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 306s 6ms/step - loss: 0.0707 - acc: 0.9737 - val_loss: 0.3346 - val_acc: 0.9266

Epoch 00001: val_acc improved from -inf to 0.92660, saving model to /gdrive/My Drive/CIFAR_10/__24__weights.01-0.93.hdf5


# 9247

In [9]:

model.load_weights("/gdrive/My Drive/CIFAR_10/__24__weights.17-0.93.hdf5")
sgd = keras.optimizers.SGD(lr=0.01 ,decay = 1e-6 ,   momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])



reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                              factor=0.1, 
                                              patience=1, 
                                              verbose=1, 
                                              mode='auto', 
#                                               min_delta=0.0001, 
#                                               cooldown=0, 
                                              min_lr=0.001)

      
  
# lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/__24__weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=1)


csv_logger = keras.callbacks.CSVLogger(filename =  "180-250-epoch__.csv", separator=',', append=False)


model.fit(x=x_train, 
          y= y_train, 
          callbacks=[ csv_logger , model_checkpoint],
          batch_size=32, 
          epochs=1, 
          verbose=1, 
          validation_split=0.0, 
          validation_data=(x_test, y_test), 
          shuffle=True)
         



Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 351s 7ms/step - loss: 0.0909 - acc: 0.9676 - val_loss: 0.3337 - val_acc: 0.9247

Epoch 00001: val_acc improved from -inf to 0.92470, saving model to /gdrive/My Drive/CIFAR_10/__24__weights.01-0.92.hdf5


# 9275

In [10]:

model.load_weights("/gdrive/My Drive/CIFAR_10/__24__weights.17-0.93.hdf5")
sgd = keras.optimizers.SGD(lr=0.01 ,decay = 1e-6 ,   momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])



reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                              factor=0.1, 
                                              patience=1, 
                                              verbose=1, 
                                              mode='auto', 
#                                               min_delta=0.0001, 
#                                               cooldown=0, 
                                              min_lr=0.001)

      
  
# lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/__24__weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=1)


csv_logger = keras.callbacks.CSVLogger(filename =  "180-250-epoch__.csv", separator=',', append=False)


model.fit(x=x_train, 
          y= y_train, 
          callbacks=[ csv_logger , model_checkpoint],
          batch_size=128, 
          epochs=1, 
          verbose=1, 
          validation_split=0.0, 
          validation_data=(x_test, y_test), 
          shuffle=True)
         



Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 280s 6ms/step - loss: 0.0623 - acc: 0.9781 - val_loss: 0.3314 - val_acc: 0.9275

Epoch 00001: val_acc improved from -inf to 0.92750, saving model to /gdrive/My Drive/CIFAR_10/__24__weights.01-0.93.hdf5


# 9301

In [12]:

model.load_weights("/gdrive/My Drive/CIFAR_10/__24__weights.17-0.93.hdf5")
sgd = keras.optimizers.SGD(lr=0.01 ,decay = 1e-6 ,   momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])



reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                              factor=0.1, 
                                              patience=1, 
                                              verbose=1, 
                                              mode='auto', 
#                                               min_delta=0.0001, 
#                                               cooldown=0, 
                                              min_lr=0.001)

      
  
# lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/__24__weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=1)


csv_logger = keras.callbacks.CSVLogger(filename =  "180-250-epoch__.csv", separator=',', append=False)


model.fit(x=x_train, 
          y= y_train, 
          callbacks=[ csv_logger , model_checkpoint],
          batch_size=200, 
          epochs=1, 
          verbose=1, 
          validation_split=0.0, 
          validation_data=(x_test, y_test), 
          shuffle=True)
         



Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 265s 5ms/step - loss: 0.0623 - acc: 0.9784 - val_loss: 0.3289 - val_acc: 0.9301

Epoch 00001: val_acc improved from -inf to 0.93010, saving model to /gdrive/My Drive/CIFAR_10/__24__weights.01-0.93.hdf5


# Load 9301% model 

In [14]:

model.load_weights("/gdrive/My Drive/CIFAR_10/__24__weights.01-0.93.hdf5")
sgd = keras.optimizers.SGD(lr=0.01 ,decay = 1e-6 ,   momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])



reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                              factor=0.1, 
                                              patience=1, 
                                              verbose=1, 
                                              mode='auto', 
#                                               min_delta=0.0001, 
#                                               cooldown=0, 
                                              min_lr=0.001)

      
  
# lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/__24__weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=1)


csv_logger = keras.callbacks.CSVLogger(filename =  "180-250-epoch__.csv", separator=',', append=False)


model.fit(x=x_train, 
          y= y_train, 
          callbacks=[ csv_logger , model_checkpoint],
          batch_size=250, 
          epochs=3, 
          verbose=1, 
          validation_split=0.0, 
          validation_data=(x_test, y_test), 
          shuffle=True)
         



Train on 50000 samples, validate on 10000 samples
Epoch 1/3
50000/50000 [==============================] - 257s 5ms/step - loss: 0.0586 - acc: 0.9792 - val_loss: 0.3304 - val_acc: 0.9289

Epoch 00001: val_acc improved from -inf to 0.92890, saving model to /gdrive/My Drive/CIFAR_10/__24__weights.01-0.93.hdf5
Epoch 2/3
50000/50000 [==============================] - 243s 5ms/step - loss: 0.0578 - acc: 0.9792 - val_loss: 0.3318 - val_acc: 0.9290

Epoch 00002: val_acc improved from 0.92890 to 0.92900, saving model to /gdrive/My Drive/CIFAR_10/__24__weights.02-0.93.hdf5
Epoch 3/3
50000/50000 [==============================] - 243s 5ms/step - loss: 0.0581 - acc: 0.9790 - val_loss: 0.3339 - val_acc: 0.9282

Epoch 00003: val_acc did not improve from 0.92900
